In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os


from keras.preprocessing.image import load_img,img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from keras.models import Model,Sequential
#from keras.optimizers import Adam,SGD,RMSprop

In [10]:
input_size=48
folder_path="../input/fer2013/"

In [11]:
expression="disgust"
plt.figure(figsize=(12,12))
for i in range(1,10,1):
    plt.subplot(3,3,i)
    img=load_img(folder_path+"train/"+expression + "/"+ os.listdir(folder_path+"train/"+expression)[i],target_size=(input_size,input_size))
    plt.imshow(img)
plt.show()

In [12]:
batch_size=128
train_datagen=ImageDataGenerator(rescale=1/255,zoom_range=0.2,horizontal_flip=True)
val_datagen=ImageDataGenerator(rescale=1/255)
train_set=train_datagen.flow_from_directory("../input/fer2013/train",target_size=(input_size,input_size),color_mode="grayscale",batch_size=batch_size,class_mode='categorical',shuffle=True)
test_set=val_datagen.flow_from_directory("../input/fer2013/test",target_size=(input_size,input_size),color_mode="grayscale",batch_size=batch_size,class_mode='categorical',shuffle=False)

In [13]:
#model building
model=Sequential()

model.add(Conv2D(64,(3,3),padding="same",input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),padding="same"))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Conv2D(512,(3,3),padding="same"))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


model.add(Dense(7,activation="softmax",))

model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])




In [14]:
model.summary()



In [15]:
#callbacks
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

checkpoint=ModelCheckpoint("./model.h5",monitor="val_acc",verbose=1,save_best_only=True,mode=max)

early_stopping=EarlyStopping(monitor='val_loss',min_delta=0,patience=3,verbose=1,restore_best_weights=True
                            )
reduce_learning_rate=ReduceLROnPlateau(monitor="val_loss",factor=0.2,patience=3,verbose=1,min_delta=0.0001)

callbacks_list = [early_stopping,checkpoint,reduce_learning_rate]




In [16]:
model.fit(train_set,steps_per_epoch=train_set.n/train_set.batch_size,epochs=15,validation_data=test_set,validation_steps=test_set.n/test_set.batch_size,callbacks=callbacks_list)

In [20]:
model.save("./model.h5")